In [1]:
import torch
import torch.nn as nn
import numpy as np

# from utils.adv_trainer import *

import os
import sys
root = os.path.abspath('/home/mbeliaev/home/code/robust-l0/')
sys.path.append(root)

from utils.models import *
from utils.helpers import *
from utils.attack import *

from tqdm import trange
import pickle
import json

import torch

In [2]:
beta = 3

mnist_1 = prep_data(root,256,'MNIST',beta=1)
mnist_100 = prep_data(root,256,'MNIST',beta=100)



In [11]:
img = mnist_1['x_test'][0]
print(img.min(),img.max())

img = mnist_100['x_test'][0]
print(img.min(),img.max())

tensor(0.) tensor(1.)
tensor(0.4950) tensor(0.5050)


In [12]:
in_ch = 1
device = "cuda:0"
b_norm_1 = nn.BatchNorm2d(in_ch).to(device)
b_norm_100 = nn.BatchNorm2d(in_ch).to(device)

In [17]:
scaled_img_1 = b_norm_1(mnist_1['x_test'][20].to(device))
print('beta_1: ',scaled_img_1.min(),scaled_img_1.max())

scaled_img_100 = b_norm_100(mnist_100['x_test'][20].to(device))
print('beta_100: ',scaled_img_100.min(),scaled_img_100.max())

beta_1:  tensor(-0.4279, device='cuda:0', grad_fn=<MinBackward1>) tensor(2.7880, device='cuda:0', grad_fn=<MaxBackward1>)
beta_100:  tensor(-0.3000, device='cuda:0', grad_fn=<MinBackward1>) tensor(1.9548, device='cuda:0', grad_fn=<MaxBackward1>)


In [18]:
print(scaled_img_1.mean(),scaled_img_100.mean())
print(scaled_img_1.std(),scaled_img_100.std())

tensor(5.0481e-08, device='cuda:0', grad_fn=<MeanBackward0>) tensor(1.8557e-06, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.0000, device='cuda:0', grad_fn=<StdBackward0>) tensor(0.7011, device='cuda:0', grad_fn=<StdBackward0>)


In [ ]:
# evaluate accuracy and r acc
def test_model(net_path, x, y, n_queries,  device):
    # first load the x, y
    input_shape = x[0].shape
    config = json.load(open(net_path+'/setup.json','rb'))

    net = Net(cfg_name=config['cfg_name'],
        k = config['k'],
        input_shape=input_shape,
        trunc_type=config['trunc_type'])

    net.to(device)
    net.load_state_dict(torch.load(net_path+'/net.pth', map_location=device))
    net.eval()

    # next evaluate accuracy
    acc = evaluate(net, x, y, device)
    print("Clean accuracy of netwrk: %.2f"%acc,"%")

    # and robust accuracy
    r_acc, _, _, _ = attack(net,config['perturb'],x,y,device=device,n_queries=n_queries,n_restarts=1,log_path='log.txt')
    print("Robust accuracy of netwrk: %.2f"%r_acc,"%")


In [ ]:
import pickle
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
# colors used
ORANGE = '#FF9132'
TEAL = '#0598B0'
GREEN = '#008F00'
PURPLE = '#8A2BE2'
GRAY = '#969696'
FIG_WIDTH = 9
FIG_HEIGHT = 5
FIG_NAME = 'analysis'
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 12,
    "axes.titlesize": 10,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "lines.linewidth": 2
})


In [ ]:
first_mnist = np.array(mnist_simp.trunc[1].weight.data.detach().cpu())
print(first_mnist.shape)

first_cifar = np.array(cifar_simp.trunc[1].weight.data.detach().cpu())
print(first_cifar.shape)

n_rows = 1
n_cols = 2
n_bins = 10

fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
gs = GridSpec(n_rows, n_cols, figure=fig)
gs.update(wspace=0.2, hspace=0.5)
# First lets make the 5 population charts
mnist_axs = fig.add_subplot(gs[0,0])
cifar_axs = fig.add_subplot(gs[0,1])
fig.suptitle("Distribtion of Truncation Layer Weights")

# mnist first
mnist_axs.hist(first_mnist,bins=n_bins,density=False)
mnist_axs.set_title("MNIST")
cifar_axs.hist(first_cifar,bins=n_bins,density=False)
cifar_axs.set_title("CIFAR") 


mnist_axs.set_xlim(0,1)
cifar_axs.set_xlim(0,1)
# hist, edges = np.histogram(first_mnist, bins=n_bins, density=True)
# print(edges)
# print(edges[1:]-edges[0:-1])
# print(np.arange(edges[0],edges[1],10))
# axs = [fig.add_subplot(gs[i//n_cols,i%n_cols]) for i in range(n_cols*n_rows)]
# for i in range(len(axs)):
#     axs[i].set_title('expert %d'%i)
#     hist, edges = np.histogram(expert_l2[i], bins=n_bins, density=True)
#     # print(edges[1:]-edges[0:-1])
#     # print(np.arange(edges[0],edges[1],10))
#     axs[i].bar(x=edges[0:-1],height=hist,align='edge',width=edges[1:]-edges[0:-1])
#     axs[i].set_xticks(edges[0:100:10])
#     axs[i].set_xticklabels(np.round(edges[0:100:10],decimals=1))
#     # let us also plot the MLE of the Gamma Dist
#     temp_s = np.log(np.mean(expert_l2[i])) - np.mean(np.log(expert_l2[i]))
#     alphas[i] = (3-temp_s+np.sqrt((3-temp_s**2)+24*temp_s))/(12*temp_s)
#     scales[i] = np.mean(expert_l2[i])/alphas[i]
#     pdf = stats.gamma.pdf(x=edges, a=alphas[i], scale = scales[i])
#     axs[i].plot(edges, pdf, c='r', label=r'$\alpha=%.2f, \beta=%.2f$'%(alphas[i],1/scales[i]))
#     # mean = np.mean(expert_l2[i])
#     # std = np.std(expert_l2[i])
#     # # axs[i].axvline(x=mean,c='r',label='mean')
#     # # axs[i].axvline(x=std,c='g',label='std')
#     axs[i].set_xlim(0,max(expert_l2[i]))
#     axs[i].legend()
# axs[0].legend()

    # print(hist)
# for i in range(len(ax_bars)):
#     ax_bars[i].set_xlim(-0.5,9.5)
#     ax_bars[i].set_ylim(0,1)
#     ax_bars[i].set_xticks([])
#     ax_bars[i].set_yticks([])
#     ax_bars[i].bar(np.arange(10),all_noise_levels[i])


# ax_bars[6].set_xticks([0,1,2,3,4,5,6,7,8,9])
# ax_bars[6].set_xticklabels([1,2,3,4,5,6,7,8,9,10])
# ax_bars[6].set_xlabel('Annotator')

# ax_bars[3].set_yticks([0,0.5,1])
# ax_bars[3].set_yticklabels([0.0,0.5,1.0])
# ax_bars[3].set_ylabel('Expertise Level')

In [ ]:
first_mnist = np.array(mnist_simp.trunc[1].weight.data.detach().cpu())
print(first_mnist.shape)

first_cifar = np.array(cifar_simp.trunc[1].weight.data.detach().cpu())
print(first_cifar.shape)

n_rows = 1
n_cols = 2

fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
gs = GridSpec(n_rows, n_cols, figure=fig)
gs.update(wspace=0.2, hspace=0.5)
# First lets make the 5 population charts
mnist_axs = fig.add_subplot(gs[0,0])
cifar_axs = fig.add_subplot(gs[0,1])
fig.suptitle("Value of Truncation Layer Weights")

# mnist first
img_mnist = mnist_axs.imshow(np.reshape(first_mnist,(28,28)),cmap='Greens')
img_mnist.set_clim(0,1)
mnist_axs.set_title("MNIST")
cbar = mnist_axs.figure.colorbar(img_mnist, ax=mnist_axs)
# cifar next
img_cifar = cifar_axs.imshow(np.reshape(first_cifar[0:32*32],(32,32)),cmap='Blues')
img_cifar.set_clim(0,1)
cifar_axs.set_title("CIFAR (1st channel)")
cbar = cifar_axs.figure.colorbar(img_cifar, ax=cifar_axs)

    # fig, ax = plt.subplots()
    # im = ax.imshow(prosocial_percentages, cmap='Greens')
    # im.set_clim(0, 1)
    # ax.set_xticks(np.arange(db))
    # ax.set_yticks(np.arange(db))
    # # ... and label them with the respective list entries
    # ax.set_xticklabels(b0)
    # ax.set_yticklabels(sorted(b1, reverse=True)) # Reverse the y labels
    # ax.set_xlabel('$x_1$')
    # ax.set_ylabel('$y_1$')
    # cbar = ax.figure.colorbar(im, ax=ax)
    # cbar.ax.set_ylabel('Percentage of Prosocial Runs', rotation=-90, va="bottom")
# mnist_axs.set_xlim(0,1)
# cifar_axs.set_xlim(0,1)

Now lets look at the distribution of the actual dataset

In [ ]:
def get_pixel_data(i,data):
    pixels = []
    for batch in data:
        for img in batch:
            pixels.append(img.flatten()[i].item())
    pixels = np.asarray(pixels)
    return pixels.mean(), pixels.std()

def get_pixel_raw(i,data):
    pixels = []
    for batch in data:
        for img in batch:
            pixels.append(img.flatten()[i].item())
    pixels = np.asarray(pixels)
    return pixels

def get_dataset_raw(data):
    num_pixels = data[0][0].numel()
    out = get_pixel_raw(0, data)
    for px in range(1, num_pixels):
        out = np.concatenate((out,get_pixel_raw(px, data)))

    return out

def get_dataset_stats(data):
    ch = data[0][0].shape[0]
    l = data[0][0].shape[1]
    w = data[0][0].shape[2]
    means = np.zeros((ch,l,w))
    stds = np.zeros((ch,l,w))

    for i_ch in range(ch):
        for i_l in range(l):
            for i_w in range(w):
                i_pixel = i_ch*(l*w) + i_l*w + i_w
                means[i_ch,i_l,i_w], stds[i_ch,i_l,i_w] = get_pixel_data(i_pixel, data)

    
    return means, stds

In [ ]:
first_mnist = np.array(mnist_simp.trunc[1].weight.data.detach().cpu())
print(first_mnist.shape)

first_cifar = np.array(cifar_simp.trunc[1].weight.data.detach().cpu())
print(first_cifar.shape)

n_rows = 1
n_cols = 2

fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
gs = GridSpec(n_rows, n_cols, figure=fig)
gs.update(wspace=0.2, hspace=0.5)
# First lets make the 5 population charts
mnist_axs = fig.add_subplot(gs[0,0])
cifar_axs = fig.add_subplot(gs[0,1])
fig.suptitle("Average value of pixels in dataset")

# mnist first
means, _ = get_dataset_stats(mnist['x_test'])
img_mnist = mnist_axs.imshow(means[0],cmap='Greens')
img_mnist.set_clim(0,1)
mnist_axs.set_title("MNIST")
cbar = mnist_axs.figure.colorbar(img_mnist, ax=mnist_axs)
# cifar next
means, _ = get_dataset_stats(cifar['x_test'])
img_cifar = cifar_axs.imshow(means[0],cmap='Blues')
img_cifar.set_clim(0,1)
cifar_axs.set_title("CIFAR (1st channel)")
cbar = cifar_axs.figure.colorbar(img_cifar, ax=cifar_axs)

In [ ]:
first_mnist = np.array(mnist_simp.trunc[1].weight.data.detach().cpu())
print(first_mnist.shape)

first_cifar = np.array(cifar_simp.trunc[1].weight.data.detach().cpu())
print(first_cifar.shape)

n_rows = 1
n_cols = 2
n_bins = 10

fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
gs = GridSpec(n_rows, n_cols, figure=fig)
gs.update(wspace=0.2, hspace=0.5)
# First lets make the 5 population charts
mnist_axs = fig.add_subplot(gs[0,0])
cifar_axs = fig.add_subplot(gs[0,1])
fig.suptitle("Distribtion Pixel Values")

# mnist first
data = get_dataset_raw(mnist['x_test'][0:20])
mnist_axs.hist(data,bins=n_bins,density=False)
mnist_axs.set_title("MNIST")

# cifar next
data = get_dataset_raw(cifar['x_test'][0:20])
cifar_axs.hist(data,bins=n_bins,density=False)
cifar_axs.set_title("CIFAR") 


mnist_axs.set_xlim(0,1)
cifar_axs.set_xlim(0,1)

In [ ]:
data = get_dataset_raw(mnist['x_test'][0:20])

In [ ]:
print("%.2f"%(100*(data==0).sum()/len(data)),"%","of data is zero")
print("%.2f"%(100*(data==1).sum()/len(data)),"%","of data is one")

In [ ]:
# how many ones are there per image? 
num_ones = []
for batch in mnist['x_test']:
    for img in batch:
        num_ones.append((img==1).sum().item())

num_zeros = []
for batch in mnist['x_test']:
    for img in batch:
        num_zeros.append((img==0).sum().item())

print('On average %.2f'%(np.asarray(num_ones).mean()/(.28*28)),"%","of pixels are one",np.asarray(num_ones).mean())
print('On average %.2f'%(np.asarray(num_zeros).mean()/(.28*28)),"%","of pixels are zero",np.asarray(num_zeros).mean())

In [ ]:
# how many ones are there per image? 
num_ones = []
for batch in cifar['x_test']:
    for img in batch:
        num_ones.append((img==1).sum().item())

num_zeros = []
for batch in cifar['x_test']:
    for img in batch:
        num_zeros.append((img==0).sum().item())

print('On average %.2f'%(np.asarray(num_ones).mean()/(.32*32*3)),"%","of pixels are one",np.asarray(num_ones).mean())
print('On average %.2f'%(np.asarray(num_zeros).mean()/(.32*32*3)),"%","of pixels are zero",np.asarray(num_zeros).mean())